In [1]:
import importlib
import subprocess
import sys
import os

def check_and_install_packages(package_list):
    missing_packages = []
    
    for package in package_list:
        try:
            importlib.import_module(package)
        except ImportError:
            if package == 'lammps_logfile':
                missing_packages.append('git+https://github.com/henriasv/lammps-logfile.git')
            elif package == 'import_ipynb':
                missing_packages.append('import-ipynb')
            else:
                missing_packages.append(package)

    if missing_packages:
        print(f"Missing packages: {', '.join(missing_packages)}")
        install_command = ["pip", "install"] + missing_packages
        subprocess.run(install_command)
        print("Packages installed successfully.")
    else:
        print("All required packages are already installed.")

all_packages = ['import_ipynb', 'scipy', 'PySimpleGUI', 'numpy', 'math', 'matplotlib', 'ase', 'lammps_logfile']
check_and_install_packages(all_packages)

import import_ipynb
try:
    import utils as uts
    import plotting as plg
    import gui as gui
except ModuleNotFoundError:
    current_directory = os.getcwd()
    sys.path.append(current_directory)
    import utils as uts
    import plotting as plg
    import gui as gui  
import PySimpleGUI as sg
import numpy as np
import math
import PySimpleGUI as sg
import matplotlib.widgets as wg
from matplotlib.colors import is_color_like

All required packages are already installed.
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from plotting.ipynb
importing Jupyter notebook from gui.ipynb


In [2]:
# a dictionary containing tuples (log, unit_types), keys are files' absolute paths
log_unit_types_dict = {}
file_list = []  # absolute paths to loaded files
fit_cursor_dict = {}
rdf_cursor_dict = {}
view_cursor_dict = {}

sg.theme("DarkBlue7")


window = gui.make_window()
window.maximize()
window.bring_to_front()

color = window.BackgroundColor

while True:
    event, values = window.read()

    # Handles closing windows.
    if event == sg.WIN_CLOSED:    
        break

    # Handles 'Load Log Files...' file browser by loading all selected files.
    elif event == 'file_log':
        file_names = values['file_log'].split(';')
        for file_name in file_names:
            with open(file_name, 'r') as f:
                log, unit_types = uts.read_log_file(f)

                # Check if data was loaded properly.
                if log.get_keywords() == [] and unit_types == []:
                    sg.popup_ok(
                        f'An error occurred while attempting to load\n{file_name}.', title='Error')

                else:
                    # Handle case in which selected file was alredy loaded
                    if file_name in file_list:
                        window[file_name].update(visible=True)
                        log_unit_types_dict.update(
                            {file_name: (log, unit_types)})
                        uts.setup_tab(log.get_keywords(), len(
                            unit_types), window, file_name, plg.plotting_rdf_data, plg.plotting_fit_data, plg.plotting_view_data)
                        window[file_name].select()

                    # Handle case in which a new file is loaded
                    else:
                        file_list.append(file_name)
                        tab_name = file_name.split('/')[-1]
                        log_unit_types_dict.update(
                            {file_name: (log, unit_types)})
                        tab = gui.make_new_tab(tab_name, file_name)
                        window['tab_group'].add_tab(tab)
                        window[f'fig_cv_overview_{file_name}'].update(
                            background_color=color) #TODO: fix this
                        uts.setup_tab(log.get_keywords(), len(
                            unit_types), window, file_name, plg.plotting_rdf_data, plg.plotting_fit_data, plg.plotting_view_data)
                        window[file_name].select()


    # Handles 'Plot' buttons from all overview tabs
    elif 'plot_overview' in event:
        file_id = event.replace('plot_overview_', '')
        size = window[f'fig_cv_overview_{file_id}'].get_size()
        fig_cv = window[f'fig_cv_overview_{file_id}'].TKCanvas
        controls_cv = window[f'controls_cv_overview_{file_id}'].TKCanvas
        plg.plot_overview(log_unit_types_dict, fig_cv, controls_cv, values, file_id, size)


    # Handles 'Load RDF files...' files browser
    elif 'file_rdf' in event:
        # retrieve name of the log file
        file_id = event.replace('file_rdf_', '')
        file_names = values[event].split(';')
        for file_name in file_names:
            with open(file_name, 'r') as f:
                uts.rdf_dict, rdf_cols_number = uts.read_RDF_file(f, file_id)

        combo_values = [i for i in range(1, rdf_cols_number-1)]
        window[f'combo_col_rdf_{file_id}'].update(
            disabled=False, values=combo_values, value=combo_values[0])
        window[f'clear_rdf_data_{file_id}'].update(
            disabled=False)

    elif 'file_view' in event:
        # retrieve name of the log file
        file_id = event.replace('file_view_', '')
        file_names = values[event].split(';')
        window[f'file_view_{file_id}'].set_cursor('watch')
        for file_name in file_names:
            with open(file_name, 'r') as f:
                uts.view_dict = uts.read_dump_file(f, file_id)
        #window[f'fig_cv_view_{file_id}'].set_cursor('arrow')
        window[f'clear_dump_data_{file_id}'].update(
            disabled=False)
        #uts.update_view_controls(window, file_id, values)

    # Handles clicking on the fit table
    elif '+CLICKED+' in event:
        file_id = event[0].replace('table_fit_', '')
        selected_rows = plg.selected_rows_dict[file_id]
        clicked_row = event[2][0]
        if clicked_row in selected_rows:
            selected_rows.remove(clicked_row)
            new_selected_rows = selected_rows
        elif len(selected_rows) > 0:
            new_selected_rows = [selected_rows[-1], clicked_row]
        else:
            new_selected_rows = [clicked_row]
        plg.selected_rows_dict[file_id] = new_selected_rows
        #if None not in new_selected_rows:
            #window[f'table_fit_{file_id}'].update(select_rows=new_selected_rows)

    elif 'intersection' in event:
        file_id = event.replace('intersection_', '')
        xq = values[f'combo_x_fit_{file_id}']
        n = values[f'combo_n_fit_{file_id}'] - 1
        yq = values[f'combo_y_fit_{file_id}']
        log, unit_types = log_unit_types_dict[file_id]
        x = log.get(xq, n)
        y = log.get(yq, n)
        u_x = uts.units(unit_types[n], xq)
        selected_rows = plg.selected_rows_dict[file_id]
        if len(selected_rows) == 2:
            row1, row2 = selected_rows
            slope1, intercept1 = plg.plotting_fit_data[file_id]['table_rows'][row1][1:3]
            slope2, intercept2 = plg.plotting_fit_data[file_id]['table_rows'][row2][1:3]
            x_i, y_i = uts.intersection(slope1, intercept1, slope2, intercept2)
            if math.isnan(x_i):
                sg.popup_ok(
                    f'Error occurred while calculating intersection point. Lines are almost parallel.', title='Warning')
            window[f'intersection_value_{file_id}'].update(
                value=f'{uts.format_number(x_i)} {u_x}')
            intersection_line = plg.plotting_fit_data[file_id]['intersection_line']
            if intersection_line:
                line = intersection_line.pop(0)
                line.remove()
                plg.plotting_fit_data[file_id]['intersection_line'] = None
                plg.plotting_fit_data[file_id]['fit_fig'].canvas.draw()
            if x_i <= x.max() and x_i >= x.min():
                ax_fit = plg.plotting_fit_data[file_id]['fit_axs']
                plg.plotting_fit_data[file_id]['intersection_line'] = ax_fit.plot(
                    [x_i, x_i], [y.min(), y.max()], linestyle='dashed', color='red')
                plg.plotting_fit_data[file_id]['fit_fig'].canvas.draw()
        else:
            sg.popup_ok(
                f'Before calculating intersection point make sure two lines from the table are selected.', title='Warning')

    elif 'plot_fit' in event:
        file_id = event.replace('plot_fit_', '')
        fig_cv = window[f'fig_cv_fit_{file_id}'].TKCanvas
        controls_cv = window[f'controls_cv_fit_{file_id}'].TKCanvas
        size = window[f'fig_cv_fit_{file_id}'].get_size()
        axs = plg.plot_fit(log_unit_types_dict, fig_cv, controls_cv,
                           values, file_id, window, size)
        fit_cursor_dict[file_id] = wg.Cursor(axs, useblit=True, color='red',
                                            horizOn=False)

    # Handles 'Plot' buttons in analyze windows.
    elif 'plot_rdf' in event:
        file_id = event.replace('plot_rdf_', '')
        fig_cv = window[f'fig_cv_rdf_{file_id}'].TKCanvas
        controls_cv = window[f'controls_cv_rdf_{file_id}'].TKCanvas
        size = window[f'fig_cv_rdf_{file_id}'].get_size()
        axs = plg.plot_rdf(log_unit_types_dict, fig_cv, controls_cv,
                           values, file_id, window, size)
        rdf_cursor_dict[file_id] = (wg.Cursor(
            axs[0], useblit=True, color='red'), wg.Cursor(axs[1], useblit=True, color='red'))

    elif 'plot_view' in event:
        file_id = event.replace('plot_view_', '')
        fig_cv = window[f'fig_cv_view_{file_id}'].TKCanvas
        controls_cv = window[f'controls_cv_view_{file_id}'].TKCanvas
        log, unit_types = log_unit_types_dict[file_id]
        fig, axs = plg.plot_view(log_unit_types_dict, fig_cv, controls_cv,
                            values, file_id, window)
        view_cursor_dict[file_id] = (fig, axs[1], wg.Cursor(axs[0], useblit=True, color='red'))
        window[f'replot_system_{file_id}'].update(disabled=True)

    elif 'clear_rdf_data' in event:
        file_id = event.replace('clear_rdf_data_', '')
        del uts.rdf_dict[file_id]
        window[f'combo_col_rdf_{file_id}'].update(
            disabled=True, values=[])
        window[f'clear_rdf_data_{file_id}'].update(
            disabled=True)
        
    elif 'clear_dump_data' in event:
        file_id = event.replace('clear_dump_data_', '')
        del uts.view_dict[file_id]
        window[f'clear_dump_data_{file_id}'].update(
            disabled=True)
        window[f'replot_system_{file_id}'].update(disabled=True)

    elif 'close_tab' in event:
        file_id = ''
        if 'overview' in event:
            file_id = event.replace('close_tab_overview_', '')
        elif 'fit' in event:
            file_id = event.replace('close_tab_fit_', '')
        elif 'view' in event:
            file_id = event.replace('close_tab_view_', '')
        elif 'rdf' in event:
            file_id = event.replace('close_tab_rdf_', '')    
        window[file_id].update(visible=False)
        del log_unit_types_dict[file_id]

    elif 'replot_system' in event:
        file_id = event.replace('replot_system_', '')
        radii = values[f'input_view_radii_{file_id}']
        colors = values[f'input_view_color_{file_id}']
        try:
            radii = float(radii)
            if radii < 0.1:
                radii = 0.1
                window[f'input_view_radii_{file_id}'].update(0.1)
            elif radii > 8.0:
                radii = 8.0
                window[f'input_view_radii_{file_id}'].update(8.0)
            
            box_size = values[f'slider_view_size_{file_id}']
            rotx = values[f'slider_view_x_{file_id}']
            roty = values[f'slider_view_y_{file_id}']
            rotz = values[f'slider_view_z_{file_id}']
            x_center = values[f'slider_view_center_x_{file_id}']
            y_center = values[f'slider_view_center_y_{file_id}']
            z_center = values[f'slider_view_center_z_{file_id}']
            plg.plotting_view_data[file_id]['settings'] = (radii, colors, box_size, rotx, roty, rotz, x_center, y_center, z_center)
            fig = view_cursor_dict[file_id][0]
            axs = view_cursor_dict[file_id][1]
            atoms = plg.plotting_view_data[file_id]['atoms']
            atoms = uts.choose_atoms(atoms, np.array([x_center, y_center, z_center]), float(box_size))
            
            colors = colors.split(',')
            numbers = []
            atom_colors = []
            final_colors = []
            for i in range(len(colors)):
                color = colors[i].split(':')
                numbers.append(int(color[0]))
                if len(color) == 2 and is_color_like(color[1]):
                    atom_colors.append(color[1])
                else:
                    raise ValueError
            
            sorted_numbers = numbers.copy()
            sorted_numbers.sort()
            atomic_numbers = list(set(atoms.get_atomic_numbers()))
            atomic_numbers.sort()
            if sorted_numbers == atomic_numbers:
                print(numbers, atomic_numbers)
                for i in range(len(atoms)):
                    final_colors.append(atom_colors[np.where(atoms.numbers[i] == numbers)[0][0]])
                    # rewrite upper line below considering atomic_numbers in ndarray

            else:
                raise ValueError
            
            axs.cla()
            axs.set_axis_off()
            plg.plot_atoms(atoms, axs, radii=radii, rotation=f'{rotx}x,{roty}y,{rotz}z', colors=final_colors)
            fig.canvas.draw()

        except ValueError:
            sg.popup_ok(
                'Atom Radii and Subbox Size have to be a number.\nAtom colors have to be of format: 1:red,2:lightgreen,3:blue', title='Warning')
            

window.close()

['red', 'green'] [1, 2]
[1, 2] [1, 2]
['red', 'lightgreen'] [1, 2]
['red', 'lightgreen'] [1, 2]
['red', 'lightgreen'] [1, 2]
['red', 'lightgreen'] [1, 2]
[1, 2] [1, 2]
